# Workshop 2 - Path Planning of a UR5 Robot Using Reinforcement Learning

## Introduction of the Environment

In this workshop, our objective is to perform path planning for a UR5 robot using reinforcement learning. The UR5 environment, developed and visualized with MuJoCo, acts as a digital twin of the actual system. This environment is integrated with MLPro to utilize its RL capabilities.

The environment is quite straightforward, as demonstrated in the video below:

<img src="ur5_video.gif" alt="SegmentLocal" title="segment" width="300">

In this setup, the UR5 robot is depicted with its end effector marked by a green node. The primary focus of this environment is on pick-and-place tasks. For this workshop, due to time constraints, we are concentrating solely on the pick operation. We have disregarded the target orientation and are only concerned with the position. The target picking position is set at (0, -0.33, 0.25) meters in x, y, and z coordinates, while the robot's home position is (1.54, -1.54, 1.54, -1.54, -1.54, 0.0) radians for the joint angles.

To reduce training time, we have limited the exploration area by introducing a virtual wall, represented by a transparent blue box. If the end effector collides with this wall, the environment is reset. Additionally, an allowance of ±0.05 meters around the target position is included. The maximum joint speed is restricted to 0.5 radians per second, indicated by a red transparent box. The task is considered complete when the end effector is within this allowance.

Regarding actions, they represent the joint velocities of the UR5 robot. Given that the robot has six degrees of freedom, the action space is six-dimensional, with velocities ranging from ±0.05 radians per second.

## Tasks Descriptions
1. **Reward Shaping**: Developing a reward function to steer the RL agent towards desired behaviors more effectively by offering intermediate rewards or penalties.
2. **State Information Selection**: The process of choosing which aspects of the environment's state to include in the agent's state representation to effectively learn and make decisions.
3. **Run RL Training**: Execute the RL training setup in MLPro and analyze the results. If the goals are not met, consider revisiting and adjusting the previous two tasks.

## Executable Code

### 0. Import the related modules
The task involves importing the necessary libraries and packages.

In [1]:
import torch
from mlpro.bf.plot import DataPlotting
from mlpro.rl import *
from environment import UR5_Mujoco
from stable_baselines3 import PPO, DDPG, A2C, HerReplayBuffer
from stable_baselines3.common.noise import NormalActionNoise, OrnsteinUhlenbeckActionNoise
from mlpro_int_sb3.wrappers import WrPolicySB32MLPro
from mlpro_int_gymnasium.wrappers import WrEnvMLPro2GYM
from pathlib import Path
import numpy as np

### 1. Design your own reward function
Your first task is to create a reward function that assigns rewards and penalties to the agent for each iteration. You can design your reward function using the following three components:
1. Current End Effector Position (`current_eef`): This parameter tracks the current position of the end effector in x, y, and z coordinates.
2. Target End Effector Position (`self._target_eef`): This parameter defines the target position for the end effector in x, y, and z coordinates.
3. Target Reached (`target_reached`): This parameter indicates whether the end effector has successfully reached the target position.
4. Environment Broken (`env_broken`): This parameter signifies that the environment has been disrupted due to a collision with the virtual wall.
5. Distance between the Current and Target End Effector Positions (`distance`): This parameter measures the distance between the current and target end effector positions.

Then, the reward function should be stored in a variable named `total_rewards`.

In [2]:
class MyUR5_Mujoco(UR5_Mujoco):
    
    def _compute_reward(self, p_state_old:State, p_state_new:State) -> Reward:
        
        states_old      = p_state_old.get_values()
        states_new      = p_state_new.get_values()
        
        if self._target_orientation:
            current_eef = states_new[12:19]
        else:
            current_eef = states_new[12:15]
        
        if p_state_new.get_success():
            target_reached = True
        else:
            target_reached = False
            
        if p_state_new.get_broken():
            env_broken = True
        else:
            env_broken = False
            
        distance = -(np.linalg.norm(self._target_eef-current_eef))
            
        # Insert your reward function in this section
        total_rewards = 0
            
        reward = Reward()
        reward.set_overall_reward(total_rewards)
        
        return reward

### 2. State information selection

Your second task is to decide which state information is relevant for the RL agent to use. The environment provides five different sets of state information:
S1. Current joint angles of the robot
S2. Current joint velocity of the robot
S3. Current end effector position
S4. Current end effector orientation
S5. Current end effector velocity

You can activate or deactivate each set of state information by setting the corresponding variable to `True` for activation or `False` for deactivation. Below is an example of how to activate or deactivate the state information:
```python
ss_joint_angles     = True # S1 is activated
ss_joint_velocity   = True # S2 is activated
ss_eef_position     = True # S3 is activated
ss_eef_orientation  = True # S4 is activated
ss_eef_velocity     = True # S5 is activated
```

In the default version of this code, the PPO algorithm is selected with its default parameters initialized. You are welcome to modify the parameters as needed. You can find the documentation for the embedded PPO from Stable-Baselines3: [here](https://stable-baselines3.readthedocs.io/en/master/modules/ppo.html).

In [3]:
class Scenario_UR5_Mujoco(RLScenario):
    C_NAME = 'UR5_Mujoco'

    def _setup(self, p_mode, p_ada: bool, p_visualize: bool, p_logging) -> Model:
        
        # Select your state information in this section
        ss_joint_angles     = True
        ss_joint_velocity   = True
        ss_eef_position     = True
        ss_eef_orientation  = True
        ss_eef_velocity     = True
        
        self._env = MyUR5_Mujoco(
            p_allowance_eef_pose=0.05,
            p_velocity_max=0.5,
            p_logging=p_logging
            )
        
        policy_kwargs = dict(activation_fn=torch.nn.Tanh,
                              net_arch=dict(pi=[64, 64], vf=[64, 64]))
        policy_sb3 = PPO(
            policy="MlpPolicy",
            n_steps=2048,
            learning_rate=0.001,
            batch_size=64,
            env=None,
            _init_setup_model=False,
            policy_kwargs=policy_kwargs,
            device="cpu",
            seed=2)
        
        state_space         = self._env.get_state_space()
        list_ss             = []
        
        if ss_joint_angles:
            for ss in range(6):
                list_ss.append(state_space.get_dim_ids()[ss])
                
        if ss_joint_velocity:
            for ss in range(6):
                list_ss.append(state_space.get_dim_ids()[ss+6])
                
        if ss_eef_position:
            for ss in range(3):
                list_ss.append(state_space.get_dim_ids()[ss+12])
                
        if ss_eef_orientation:
            for ss in range(4):
                list_ss.append(state_space.get_dim_ids()[ss+15])
                
        if ss_eef_velocity:
            for ss in range(6):
                list_ss.append(state_space.get_dim_ids()[ss+19])
        
        _sspace = state_space.spawn(list_ss)

        policy_wrapped = WrPolicySB32MLPro(
            p_sb3_policy=policy_sb3,
            p_cycle_limit=self._cycle_limit,
            p_observation_space=_sspace,
            p_action_space=self._env.get_action_space(),
            p_ada=p_ada,
            p_visualize=p_visualize,
            p_logging=p_logging)

        return Agent(
            p_policy=policy_wrapped,
            p_envmodel=None,
            p_name='Smith',
            p_ada=p_ada,
            p_visualize=p_visualize,
            p_logging=p_logging)

### 3. RL Training configuration
Your next task is to initiate the RL training by executing the following code. You have the flexibility to adjust two key variables: `cycle_limit` and `cycles_per_epi_limit`. Currently, these are set to 30000 and 1000, respectively. For the initial run, it is advisable to maintain these default values. Once the training begins, a graph will appear, allowing you to monitor the training progress in real-time.

In [ ]:
cycle_limit             = 30000
cycles_per_epi_limit    = 1000
logging                 = Log.C_LOG_WE
visualize               = True
path                    = str(Path.home())
plotting                = True

training = RLTraining(
    p_scenario_cls=Scenario_UR5_Mujoco,
    p_cycle_limit=cycle_limit,
    p_cycles_per_epi_limit=cycles_per_epi_limit,
    p_path=path,
    p_visualize=visualize,
    p_logging=logging
)

training.run()

C:\Users\Yuwono\.conda\envs\workshop\lib\site-packages\gymnasium\core.py:311: UserWarning: WARN: env.data to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.data` for environment variables or `env.get_wrapper_attr('data')` that will search the reminding wrappers.
  logger.warn(


2024-09-03  14:09:34.979919  W  Training "RL": ------------------------------------------------------------------------------ 
2024-09-03  14:09:34.980960  W  Training "RL": ------------------------------------------------------------------------------ 
2024-09-03  14:09:34.980960  W  Training "RL": -- Training run 0 started... 
2024-09-03  14:09:34.980960  W  Training "RL": ------------------------------------------------------------------------------ 
2024-09-03  14:09:34.980960  W  Training "RL": ------------------------------------------------------------------------------ 
 
2024-09-03  14:09:35.005245  W  Training "RL": ------------------------------------------------------------------------------ 
2024-09-03  14:09:35.005245  W  Training "RL": -- Training episode 0 started... 
2024-09-03  14:09:35.005245  W  Training "RL": ------------------------------------------------------------------------------ 
 


C:\Users\Yuwono\.conda\envs\workshop\lib\site-packages\gymnasium\core.py:311: UserWarning: WARN: env.data to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.data` for environment variables or `env.get_wrapper_attr('data')` that will search the reminding wrappers.
  logger.warn(
C:\Users\Yuwono\AppData\Roaming\Python\Python39\site-packages\glfw\__init__.py:912: GLFWError: (65537) b'The GLFW library is not initialized'
  warnings.warn(message, GLFWError)


2024-09-03  14:09:38.543565  W  Training "RL": Limit of 1000 cycles per episode reached (Training) 
2024-09-03  14:09:38.543565  W  Training "RL": ------------------------------------------------------------------------------ 
2024-09-03  14:09:38.543565  W  Training "RL": -- Training episode 0 finished after 1000 cycles 
2024-09-03  14:09:38.543565  W  Training "RL": -- Training cycles finished: 1000 
2024-09-03  14:09:38.543565  W  Training "RL": ------------------------------------------------------------------------------ 

 
2024-09-03  14:09:38.544574  W  Training "RL": ------------------------------------------------------------------------------ 
2024-09-03  14:09:38.544574  W  Training "RL": -- Training episode 1 started... 
2024-09-03  14:09:38.544574  W  Training "RL": ------------------------------------------------------------------------------ 
 


### 4. Plotting reward values
You can execute the following lines of code to visualize the rewards generated by your function.

In [ ]:
mem = training.get_results().ds_rewards
data_printing = {mem.names[0]: [False],
                 mem.names[1]: [False],
                 mem.names[2]: [False],
                 mem.names[3]: [False],
                 mem.names[4]: [True, 0, -1]}
mem_plot = DataPlotting(mem,
                        p_showing=plotting,
                        p_printing=data_printing,
                        p_type=DataPlotting.C_PLOT_TYPE_EP_S,
                        p_window=10)
mem_plot.get_plots()